In [ ]:
reset -f


In [ ]:
import os
import pandas as pd
import pytz


In [ ]:
start_date: str = "2022-06-15"
end_date: str = "2022-10-01"
defined_timezone = pytz.timezone("Europe/Helsinki")
start_date = pd.to_datetime(start_date).tz_localize(defined_timezone)
end_date = pd.to_datetime(end_date).tz_localize(defined_timezone)
start_date = pd.to_datetime(start_date).tz_convert(pytz.utc)
end_date = pd.to_datetime(end_date).tz_convert(pytz.utc)



In [ ]:
input_folder: str = 'InputData'
preprocessed_input_folder: str = 'PreprocessedData'
defa_file_name_gzip: str = 'Defa_Table.gzip'
chargeing_point_name_file_name: str = "Chargeing_point_name.json"


In [ ]:
file_path: str = os.path.join(preprocessed_input_folder, defa_file_name_gzip)
df_defa: pd.DataFrame = pd.read_parquet(file_path)
file_path: str = os.path.join(input_folder, chargeing_point_name_file_name)
file = open(file_path, "r")
df_chargeing_point_name: pd.DataFrame = pd.read_json(file, lines=True)
file.close()


In [ ]:
df_defa: pd.DataFrame = pd.merge(df_defa, df_chargeing_point_name, left_on=["id", "connectorId"],
                                 right_on=["id", "connectorId"])
df_defa.head()


In [ ]:
def getFirstEventGroupedByAutopaikka(df, cutoff_time):
    mask = df['start_time'] > cutoff_time
    filtered_df = df_defa[mask]
    filtered_df
    df_grouppedByAutopaikka = filtered_df.groupby(["Autopaikka"])
    return df_grouppedByAutopaikka


In [ ]:
def getLatestEventGroupedByAutopaikka(df, cutoff_time):
    mask = df['end_time'] < cutoff_time
    filtered_df = df_defa[mask]
    filtered_df
    df_grouppedByAutopaikka = filtered_df.groupby(["Autopaikka"])
    return df_grouppedByAutopaikka


In [ ]:
cutoff_time = end_date
df_grouppedByAutopaikka = getLatestEventGroupedByAutopaikka(df_defa, cutoff_time)


In [ ]:
print(df_grouppedByAutopaikka[["meterStopKwh", "end_time"]].last())


In [ ]:
cutoff_time = start_date
df_grouppedByAutopaikka = getFirstEventGroupedByAutopaikka(df_defa, cutoff_time)


In [ ]:
print(df_grouppedByAutopaikka[["meterStartKwh", "start_time"]].first())


In [ ]:
df_grouppedByAutopaikka.get_group("AP7")
